In [9]:
import torch 
from torch import nn

In [13]:
net = nn.Sequential( nn.Linear( 4,8) , nn.ReLU() , nn.Linear( 8 ,1 ) )
x  = torch.ones( 2,4 )
net( x ) , net[0](x)

(tensor([[0.4425],
         [0.4425]], grad_fn=<AddmmBackward>),
 tensor([[-0.0718, -0.6553,  0.0997,  0.5128,  0.1804, -1.3132, -0.4703,  0.5067],
         [-0.0718, -0.6553,  0.0997,  0.5128,  0.1804, -1.3132, -0.4703,  0.5067]],
        grad_fn=<AddmmBackward>))

In [14]:
print( net[0].weight.shape , net[0].weight )

torch.Size([8, 4]) Parameter containing:
tensor([[-0.0276,  0.1369, -0.2098,  0.3570],
        [ 0.2935, -0.4089, -0.4007,  0.1465],
        [ 0.2554, -0.3449, -0.0934, -0.2119],
        [-0.2409,  0.4620, -0.2183,  0.3321],
        [ 0.1007, -0.1509,  0.0962,  0.4125],
        [-0.4971, -0.2493, -0.3001,  0.1739],
        [-0.3087, -0.0322,  0.3761, -0.4283],
        [ 0.4632, -0.0415, -0.0617,  0.2544]], requires_grad=True)


## **这里就会有一个问题，我们之前做的相乘都是X@Weight+bias ,但是在nn.Linear里不是这样做的，因为显然这个weight矩阵是8 * 4 ,X是2 * 4，显然不是X@Weight这么简单**
* 通过阅读nn.Linear的源码知道了在linear中做了这个操作ret = torch.addmm(bias, input, weight.t())，具体查看functional的linear函数，所以可知在线性层中是X@Weight.t()

In [24]:
x.mm( net[0].weight.detach().T ) + net[0].bias.detach()  #结果正确

tensor([[-0.0718, -0.6553,  0.0997,  0.5128,  0.1804, -1.3132, -0.4703,  0.5067],
        [-0.0718, -0.6553,  0.0997,  0.5128,  0.1804, -1.3132, -0.4703,  0.5067]])

In [25]:
net[0].weight

Parameter containing:
tensor([[-0.0276,  0.1369, -0.2098,  0.3570],
        [ 0.2935, -0.4089, -0.4007,  0.1465],
        [ 0.2554, -0.3449, -0.0934, -0.2119],
        [-0.2409,  0.4620, -0.2183,  0.3321],
        [ 0.1007, -0.1509,  0.0962,  0.4125],
        [-0.4971, -0.2493, -0.3001,  0.1739],
        [-0.3087, -0.0322,  0.3761, -0.4283],
        [ 0.4632, -0.0415, -0.0617,  0.2544]], requires_grad=True)

In [27]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.1523, -0.2268,  0.2209,  0.0400, -0.0794, -0.2174, -0.1404,  0.3296]])),
             ('bias', tensor([0.2472]))])

In [28]:
net[1] #所以在torch里要将激活函数算作一层

ReLU()

In [33]:
for block in  net.state_dict():
    print( block , net.state_dict()[block].size())

0.weight torch.Size([8, 4])
0.bias torch.Size([8])
2.weight torch.Size([1, 8])
2.bias torch.Size([1])


In [48]:
next(net.named_modules())


('',
 Sequential(
   (0): Linear(in_features=4, out_features=8, bias=True)
   (1): ReLU()
   (2): Linear(in_features=8, out_features=1, bias=True)
 ))